


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
# Load the data and preprocess data and store corpus in raw_text
import nltk
nltk.download('punkt') # For tokenizers
from nltk.tokenize import RegexpTokenizer
data=""
with open("corpus.txt") as f:
  data=f.read()
raw_text = data.lower() # changing the text to lowercase
tokenizer = RegexpTokenizer(r'[^\W_]+')
words = tokenizer.tokenize(raw_text)
raw_text = ' '.join(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Hyperparameters of the model
vocab_size = 2570 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [4]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index

In [5]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]

In [6]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  27283


In [7]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [8]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           257000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 2570)              331530    
Total params: 673,010
Trainable params: 673,010
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=150, batch_size=128, validation_split=0.2 ) 

Epoch 1/150
171/171 [==============================] - 3s 20ms/step - loss: 6.3413 - accuracy: 0.0533 - val_loss: 6.1905 - val_accuracy: 0.0823
Epoch 2/150
171/171 [==============================] - 3s 15ms/step - loss: 5.8815 - accuracy: 0.0546 - val_loss: 6.1308 - val_accuracy: 0.0823
Epoch 3/150
171/171 [==============================] - 2s 14ms/step - loss: 5.7624 - accuracy: 0.0578 - val_loss: 6.1402 - val_accuracy: 0.0837
Epoch 4/150
171/171 [==============================] - 3s 15ms/step - loss: 5.6652 - accuracy: 0.0630 - val_loss: 6.1482 - val_accuracy: 0.0861
Epoch 5/150
171/171 [==============================] - 3s 15ms/step - loss: 5.5719 - accuracy: 0.0698 - val_loss: 6.0908 - val_accuracy: 0.0924
Epoch 6/150
171/171 [==============================] - 3s 15ms/step - loss: 5.4647 - accuracy: 0.0794 - val_loss: 6.0554 - val_accuracy: 0.0990
Epoch 7/150
171/171 [==============================] - 3s 15ms/step - loss: 5.3369 - accuracy: 0.0981 - val_loss: 6.0341 - val_accuracy:

In [10]:
reverse_word_map = {v:k for k,v in word_index.items()} ## Create word to idx map using tokenizer.word_index

In [11]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
	print ("Seed: \n",  input_str)
	tokens = tokenizer.texts_to_sequences([input_str])
	next_words = []
	for i in range(n):
		prediction = model.predict( np.array(tokens) , verbose=0)[0]
		# get next word index. Use reverse_word_map to get the word
		next_word_index = np.argmax(prediction) + 1
		next_word = reverse_word_map[next_word_index]
		tokens[0].append(next_word_index)
		tokens[0].pop(0)
		next_words.append(next_word)
	final_string = ' '.join(next_words)
	return final_string

In [29]:
# pick a random seed
numpy.random.seed(0)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print("Next 10 words: \n", next_tokens( input_str , 10))

Seed: 
 let me think was i the same when i got up this morning i almost think i can remember feeling a little different but if i m not the same the next question is who in the world am i ah that s the great puzzle and she began thinking
Next 10 words: 
 why be back a he so i would not queen


In [30]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
tokens = tokenizer.texts_to_sequences([input_str])[0][:50]
input_str = ' '.join([reverse_word_map[value] for value in tokens])
print("Next 10 words: \n", next_tokens( input_str , 10))

Seed: 
 the boy laughed at the fright he had caused this time the <OOV> left angrily the <OOV> day as the boy went up the small <OOV> he suddenly saw a <OOV> <OOV> his sheep he cried as hard as he could <OOV> <OOV> <OOV> <OOV> but not a <OOV> <OOV>
Next 10 words: 
 of and and and and be and be natural as


# Character based LSTM Model 1

In [31]:
# User the preprocess data and create raw_text
data=""
with open("corpus.txt") as f:
  data=f.read()
raw_text = data.lower() # Making the text lowercase
raw_text = re.sub('_',' ',raw_text) # Replacing underscores by spaces for words like _never_
raw_text = re.sub('-',' ',raw_text) # Replacing dashes by spaces for words like rabbit-hole

# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = {ch:i for i,ch in enumerate(chars)}

In [32]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters:",n_chars)
print("Character vocab size:",n_vocab)

Total characters: 142477
Character vocab size: 44


In [33]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    # Write code here
    seq_in = [char_to_int[ch] for ch in raw_text[i:i + seq_length]]
    seq_out = char_to_int[raw_text[i + seq_length]]
    dataX.append(seq_in)
    dataY.append(seq_out)    


n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  142377


In [34]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX)

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [35]:
embedding_dim =100
max_length =100

In [36]:
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          4400      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 44)                11308     
Total params: 381,276
Trainable params: 381,276
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1113/1113 [==============================] - 20s 18ms/step - loss: 2.2024
Epoch 2/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.6962
Epoch 3/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.4984
Epoch 4/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.3819
Epoch 5/20
1113/1113 [==============================] - 19s 17ms/step - loss: 1.3028
Epoch 6/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.2426
Epoch 7/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1917
Epoch 8/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1505
Epoch 9/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1101
Epoch 10/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0783
Epoch 11/20
1113/1113 [==============================] - 19s 17ms/step - loss: 1.0472
Epoch 12/20
1113/1113 [==============================] - 20s 18

In [38]:
#implement mapping of integer to character
int_to_char = {i:ch for i,ch in enumerate(chars)}

In [39]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_string = ''
	seq_in = [char_to_int[ch] for ch in pattern]
	for i in range(x):
		# Complete Code
		prediction = model.predict(np.array([seq_in]), verbose=0)[0]
		char_index = np.argmax(prediction)
		char = int_to_char[char_index]
		seq_in.append(char_index)
		seq_in.pop(0)
		final_string += char
	return final_string

In [53]:
# pick a random seed
numpy.random.seed(30)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print("Input string: \n",input_str)
print("Next 200 chars: \n",predict_next_100_chars(input_str,200))

Input string: 
  become very white; and yet you incessantly stand on your head— do you think, at your age, it is rig
Next 200 chars: 
 ht,” said the mock turtle, “but it makes me three little girls and more to sea in the same time,” said the mock turtle, “but it makes me three little girls and more to sea in the same time,” said the 


In [54]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str = input_str[:100].lower()

print("Input string: \n",input_str) 
print("Next 200 chars: \n",predict_next_100_chars(input_str,200))

Input string: 
 the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
Next 200 chars: 
 nd the moral of that is—‘the dormouse should be the right way is to go down here in the same time,” said the mock turtle, “but it makes me three little girls and more to sea in the same time,” said th


## Character based LSTM Model 2


In [55]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [56]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2225/2225 [==============================] - 50s 22ms/step - loss: 2.1099
Epoch 2/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.5601
Epoch 3/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.3838
Epoch 4/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.2876
Epoch 5/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.2170
Epoch 6/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.1640
Epoch 7/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.1201
Epoch 8/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0843
Epoch 9/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0533
Epoch 10/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0194
Epoch 11/20
2225/2225 [==============================] - 50s 22ms/step - loss: 0.9963
Epoch 12/20
2225/2225 [==============================] - 49s 22

In [57]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_string = ''
	seq_in = [char_to_int[ch] for ch in pattern]
	for i in range(x):
		# Complete Code
		prediction = model1.predict(np.array([seq_in]), verbose=0)[0]
		char_index = np.argmax(prediction)
		char = int_to_char[char_index]
		seq_in.append(char_index)
		seq_in.pop(0)
		final_string += char
	return final_string

In [67]:
# pick a random seed
numpy.random.seed(30)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print("Input string: \n",input_str)
print("Next 200 chars: \n",predict_next_100_chars(input_str,200))

Input string: 
  become very white; and yet you incessantly stand on your head— do you think, at your age, it is rig
Next 200 chars: 
 ht,” said the duchess, “and the dormouse show you how the dormouse shower things!” alice was soon found herself up and said, “and the dormouse show you how the dormouse shower things!” alice was soon 


In [68]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str = input_str[:100].lower()

print("Input string: \n",input_str)
print("Next 200 chars: \n",predict_next_100_chars(input_str,200))

Input string: 
 the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
Next 200 chars: 
 nd the same shook its head she was now and then a great deal frightened to say “what  can  the mock turtle you can  the mock turtle yet,” said the duchess, “and the dormouse show you how the dormouse 


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** On the first sentence which is in the train data(in domain), the predictions of the models are better than on the second sentence which is unseen data(out of domain). The model outputs on train data are generally more realistic and contain fewer spelling mistakes as compared to unseen data. This is due to unseen phrases not encountered during training. 

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** The text predicted by char LSTM model2 looks more realistic and there are generally fewer spelling mistakes as compared to that predicted by char LSTM model1. The outputs for model2 are generally better than that of model1 but they are still quite nonsensical. This is because the char LSTM model2 has 2 layers  and is larger than the char LSTM model1 which has only 1 layer. So, model2 is able to learn better.

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [69]:
!pip install transformers

In [70]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [71]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [72]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [73]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [74]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [75]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [76]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)

Predict next  3  words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 4, 30522])
['p', 'vol', 'math']
p    vol    math
